# Step 1: configuration

Input your walletPrivateKey here:

In [1]:
# make sure you configure mindlakesdk, please check env.py in the same folder
# please familar these configure, and you can read https://github.com/mind-network/mind-lake-sdk-python 
import env

print("check env.walletPrivateKey:", env.walletPrivateKey[0:5])
print("check env.mindLakeAppKey:", env.mindLakeAppKey[0:5])
print("check env.MINDLAKE_GATEWAY:", env.MINDLAKE_GATEWAY)

check env.walletPrivateKey: 2a776
check env.mindLakeAppKey: Kyunq
check env.MINDLAKE_GATEWAY: https://sdk.mindnetwork.xyz/node


Install depedancy and source code

In [2]:
# please "pip install mindlakesdk" if not installed
import mindlakesdk

# please "pip install minddatapack" if not installed, or git clone the source code
from minddatapack import DataPack

# please "pip install pandas" if not installed, it is used to display structure content in the notebook
import pandas

In [3]:
# check if example_data.csv file exists in the same folder, this demo will start to load this csv file.

# check if arweave_wallet.json file exists in the same folder, this demo needs to use arweave wallet to sign to send tx.

## Step 2. Local CSV -> MindLake
- Load data from a local CSV file without the metadata file, by defining each column manually.
- You can also load from a local backup into MindLake once you are familar, then no need to define each column. 

In [4]:
fn_local_csv = './example_data.csv'

# preview local csv
df = pandas.read_csv(fn_local_csv)
df

,uid,wallet_address,register_date
0,123,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
1,124,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
2,125,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441


In [5]:
dataPack1 = DataPack(env.walletPrivateKey)
result = dataPack1.loadFromCSVFileByDefineColumn(fn_local_csv, 
    [
        DataPack.Column('uid', DataPack.DataType.int4, False),
        DataPack.Column('wallet_address', DataPack.DataType.text, True),
        DataPack.Column('register_date', DataPack.DataType.timestamp, True)
    ])
assert result, result.message
df = pandas.DataFrame(dataPack1.data)
df

,0,1,2
0,123,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
1,124,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
2,125,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441


- Save the data into a table in MindLake

In [6]:
mindlake = mindlakesdk.connect(env.walletPrivateKey, env.mindLakeAppKey, env.MINDLAKE_GATEWAY)
assert mindlake, mindlake.message

# drop the table if exists
result = mindlake.datalake.dropTable('a_new_table')
print("dropTable result:", result.message)

# upload from local csv into mindlake
result = dataPack1.saveToMindLake('a_new_table', mindlake)
assert result, result.message
print("a_new_table has been saved to MindLake.")

dropTable result: Success
a_new_table has been saved to MindLake.


you can also check this new downloaded table in https://scan.mindnetwork.xyz/account/myData

## Step 3. MindLake -> Arweave
- Load data from a table in MindLake and save to Arweave

In [7]:
dataPack2 = DataPack(env.walletPrivateKey)
# make sure if you complete Step 1, so "a_new_table" exists in mindlake 
result = dataPack2.loadFromMindByQuery('select * from "a_new_table"', mindlake)
assert result, result.message
df = pandas.DataFrame(dataPack2.data)
df

,0,1,2
0,123,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
1,124,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
2,125,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441


- Save the data into Arweave

In [8]:
# make sure you have arweave_wallet.json 
result = dataPack2.saveToArweave('test_table_encrypted.csv', 'AR', './arweave_wallet.json')
assert result, result.message
print("test_table_encrypted.csv has been saved to Arweave.\n")
itemID = result.data
print("The Arweave ID is:", itemID, '\n')
arweave_explore_url = "https://viewblock.io/arweave/tx/%s"%(itemID)
arweave_file_url = "https://arseed.web3infra.dev/%s"%(itemID)
print(f"You can check on Arweave: {arweave_explore_url}\n")
print(f"And the content on Arweave: {arweave_file_url}\n")
import requests
content = requests.get(arweave_file_url).text
print("The content is:")
print(content)

test_table_encrypted.csv has been saved to Arweave.

The Arweave ID is: OXVF56-SsxPNZ9_Zy5BTnjWPaMoWdq_OpwCiOg811-U 

You can check on Arweave: https://viewblock.io/arweave/tx/OXVF56-SsxPNZ9_Zy5BTnjWPaMoWdq_OpwCiOg811-U

And the content on Arweave: https://arseed.web3infra.dev/OXVF56-SsxPNZ9_Zy5BTnjWPaMoWdq_OpwCiOg811-U

The content is:
uid,wallet_address,register_date
123,\x9ffdb6fa20d5e41952032a372d1b749750bb2c2b76de5fdd3c56fe377fe8e18e4b1958c1fbe109e2e712779e015955e9d3abf06843b2634bed682df38e992d8c34,\x0a20c65664374908163db53cfedfea5613adc2164872e4590659f8f9da3f5d108c
124,\xd60d24a0753996459f06cc86e23e1041049e7214af09e8684863760567026a0191241d018e81a9423ee47c5244a256784d09ea9429ac65334b98afa964e388f8a8,\x743ea93bff0c3797e6b336e9e10c215344fcf0289d2d2b19eb7608a90b56417e22
125,\x660a17798b68370adf43812181df6bc5ebaa969c8d9439a322e37dc111f9f2018425d41ca83cf2251a57af2f58adcee68dbfdaf5e5307807f112cef9a3014a858a,\x1c3bed52fd032aea9b1990ed80007bce85167ec7fb2d50214dc7b0d13343b1a56e



## Step 4. Arweave -> Local File
- Load data from Arweave and save to local file

In [9]:
dataPack3 = DataPack(env.walletPrivateKey)
result = dataPack3.loadFromArweave(itemID)
assert result, result.message
print("You can see file on Arweave are encrypted, but datapack can decrypted it locally")
df = pandas.DataFrame(dataPack3.data)
df

You can see file on Arweave are encrypted, but datapack can decrypted it locally


,0,1,2
0,123,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
1,124,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441
2,125,0x79Be957bf7e3003aFd0e78f04Bacbc93D3ef2fB7,2023-07-15 02:25:32.392441


- Save the data into a local file with encryption

In [10]:
result = dataPack3.saveToLocalFile('test_localtable_encrypted.csv', False)
assert result, result.message
print("test_localtable_encrypted.csv has been saved to local file.")
df = pandas.read_csv('test_localtable_encrypted.csv')
df

test_localtable_encrypted.csv has been saved to local file.


,uid,wallet_address,register_date
0,123,\xf172db5115f65c2398b861ac2d0464d028c74858314a...,\x7b3c1b5b94124f53257126d13a2d6e8db6545979c652...
1,124,\x1990e23c35acfa7257b91a92e9c10bc985727f5bef5f...,\x344cdab6ad9ba1b781c013cc849ba43c5d78f8d36df0...
2,125,\xd2ba460636b46db5091d73db1700991c165622e972c5...,\x1ef347e400cb6f3a6d763f57e93c772d0ae8c3704b77...
